In [2]:
import pandas as pd
import numpy as np
import os

# --- [1] 경로 및 파일 설정 ---
DATA_PATH = "../../data/raw/"
SUB_PATH = "./submissions/"

# 모델별 Submission 파일 (방향: Small=Voted)
M1_SUB = "0130-1923.csv"
M7_SUB = "m7_Refined_test_preds.csv"
M8_SUB = "28_m8_ft_0.76664.csv"

# --- [2] 테스트 데이터 로드 및 랭크 변환 ---
# Small=Voted(1.x)를 Large=Voted(0.x) 확률값 방향으로 변환하여 랭크 계산
m1_test = (2.0 - pd.read_csv(f"{SUB_PATH}{M1_SUB}")['voted']).rank(pct=True).values
m7_test = (2.0 - pd.read_csv(f"{SUB_PATH}{M7_SUB}")['voted']).rank(pct=True).values
# m8은 방향 보정(1 - rank) 필수
m8_test = (1 - (2.0 - pd.read_csv(f"{SUB_PATH}{M8_SUB}")['voted']).rank(pct=True)).values

# --- [3] 보수적 가중치 적용 (m8 비중 10% 제한) ---
w1, w7, w8 = 0.45, 0.45, 0.10

final_rank = (w1 * m1_test) + (w7 * m7_test) + (w8 * m8_test)

# --- [4] 최종 제출 파일 생성 ---
sub = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
sub['voted'] = 2.0 - final_rank # 최종 제출 방향 복구
output_name = f"{SUB_PATH}exp30_m1_m7_m8_Weighted_0.45_0.45_0.10.csv"
sub.to_csv(output_name, index=False)

print(f"Conservative Ensemble saved: {output_name}")

Conservative Ensemble saved: ./submissions/exp30_m1_m7_m8_Weighted_0.45_0.45_0.10.csv
